# Laboratorio 4 - Inteligencia Artificial

In [27]:
import numpy as np
from math import *
import random as rd
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Task 1.1

In [28]:
#data = np.genfromtxt("./kc_house_data.csv", delimiter=",",names=True, dtype=None, encoding=None)
data = pd.read_csv("./kc_house_data.csv")

##### Se leyó el archivo csv utilizando la librería numpy. La data esta delimitada por ','

## Task 1.2

In [29]:
Y = np.array(data["price"])
X = np.array(data["sqft_living"])


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [67]:
# se calcula la matriz 
def matriz_pol(datos, k):
    X = []
    for i in range(len(datos)):
        temp = []
        for j in range(k+1):
            temp.append(datos[i]**j)
        X.append(temp)

    X = np.array(X)
    return X

def matriz_Y(Y):
    Y = np.array(Y)
    Y = Y.reshape(len(Y), 1)
    return Y

# se calcula la matriz transpuesta
def matriz_trans(X):
    lineas = np.shape(X)[0]
    columnas = np.shape(X)[1]
    x = np.empty((columnas, lineas)) 

    for i in range(np.shape(X)[0]):
        for j in range(np.shape(X)[1]):
            x[j][i] = X[i][j]
    return x

# se crea W
def w_pol(k):
    
    W = []    
    for i in range(k+1):
        W.append(rd.randint(0, 100))        
    W = np.array(W)
    return W

## Task 1.3

Implementación vectorial del algoritmo de regresión lineal.

In [ ]:
# Building the model
def crear_modelo(X_data, Y_data, polinomio):
    x = matriz_pol(X_data, polinomio) # X de forma de matriz de grado 1
    y = matriz_Y(Y_data)
    x_trans = matriz_trans(x) # X transpuesta
    w = w_pol(polinomio) # W con valores aleatorios

    w_t = w.transpose()
    t = np.dot(x, w_t)

    # m = w[1]
    # c = w[0]

    error = 0
    L = 0.12  # The learning Rate
    epochs = 1000 # The number of iterations

    n = len(X_train)

    # Performing Gradient Descent 
    for i in range(epochs): 

        #new_w = (-2/n) * np.dot(x, w)

        # m = m - L * new_w[1]  # Update m
        # c = c - L * new_w[0]  # Update c

        w = w - L * ((-2/n) * np.dot(x, (y - np.dot(x, w))))

        error = (1/n) * (t - np.dot(x, w)).transpose() * (t - np.dot(x, w))
        
    
    return w, error

w, error = crear_modelo(X_train, Y_train, 1)
    
# print (m, c)

# W = np.array([c, m])
# Y = np.dot(x, W)

# max_indx = np.unravel_index(np.argmax(Y), Y.shape)
# min_indx = np.unravel_index(np.argmin(Y), Y.shape)

# x_maximo = x[max_indx]
# x_minimo = x[min_indx]
# y_maximo = Y[max_indx]
# y_minimo = Y[min_indx]

# x = [x_maximo[1], x_minimo[1]]
# y = [y_maximo, y_minimo]

# plt.plot(X_train, Y_train, 'o', color='black')
# plt.plot(x, y, color='red')


Y es la multiplicacion de las matrices W calculadas con valores aleatorios y de la matriz X. Con esto, se calcularon los valores maximos y minimos en Y y se obtuvieron sus valores correspondientes en X para poder graficarlo.